In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
import os
os.chdir("/content/gdrive/My Drive/CMPT413")
!ls

enron_cleaned.csv  lr.pkl      rf.pkl	text_attack.ipynb
knn.pkl		   mbayes.pkl  svm.pkl


In [5]:
!pip3 install textattack[tensorflow] nltk scikit-learn

     |████████████████████████████████| 373 kB 12.7 MB/s 
     |████████████████████████████████| 298 kB 46.0 MB/s 
     |████████████████████████████████| 769 kB 43.7 MB/s 
     |████████████████████████████████| 59 kB 7.0 MB/s 
     |████████████████████████████████| 3.1 MB 40.4 MB/s 
     |████████████████████████████████| 69 kB 7.1 MB/s 
     |████████████████████████████████| 101 kB 10.5 MB/s 
     |████████████████████████████████| 322 kB 52.0 MB/s 
     |████████████████████████████████| 124 kB 48.8 MB/s 
     |████████████████████████████████| 4.9 MB 50.0 MB/s 
     |████████████████████████████████| 454.3 MB 17 kB/s 
     |████████████████████████████████| 462 kB 45.1 MB/s 
     |████████████████████████████████| 1.2 MB 38.1 MB/s 
     |████████████████████████████████| 4.0 MB 39.8 MB/s 
     |████████████████████████████████| 4.9 MB 36.2 MB/s 
     |████████████████████████████████| 4.4 MB 38.2 MB/s 
     |████████████████████████████████| 4.3 MB 53.3 MB/s 
     |████████████

In [6]:
import sklearn
import numpy as np
import pandas as pd
import nltk
import pickle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from textattack.models.wrappers import SklearnModelWrapper
from textattack.datasets import HuggingFaceDataset, Dataset
from textattack.attack_recipes import TextFoolerJin2019
from textattack import Attacker

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:16<00:00, 28.5MB/s]
textattack: Unzipping file /root/.cache/textattack/tmp_o4j2rqv.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


In [7]:
def transform_BOW_bigram(df, column_name):
  bigram_vectorizer = CountVectorizer(ngram_range = (2, 2))
  vectFit = bigram_vectorizer.fit(df['clean_msg_no_lst'].to_numpy())
  return vectFit

In [8]:
enron_vec = pd.read_csv('enron_cleaned.csv')
enron_vec = enron_vec.fillna('')
bigram_vect_fit = transform_BOW_bigram(enron_vec, 'clean_msg_no_lst')


In [9]:
_, X_test, _, _ = train_test_split(enron_vec[['clean_msg_no_lst', 'target']], enron_vec.target, random_state = 42, test_size = 0.2)
test = []
for index, row in X_test.iterrows():
  test.append((row.clean_msg_no_lst, int(row.target)))

In [10]:
with open('./rf.pkl', 'rb') as f:
    rf = pickle.load(f)

In [11]:
print(test[1:2])
dataset = Dataset(test)
print(dataset[1:2])
model_wrapper = SklearnModelWrapper(rf, bigram_vect_fit)
# dataset = HuggingFaceDataset("sms_spam", None, "train", dataset_columns=(['sms'],'label'))
# dataset = HuggingFaceDataset(dataset, dataset_columns=(['text'],None))
attack = TextFoolerJin2019.build(model_wrapper)

attacker = Attacker(attack, dataset)
attacker.attack_dataset()

textattack: Unknown if model of class <class 'sklearn.ensemble._forest.RandomForestClassifier'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
  0%|          | 0/10 [00:00<?, ?it/s]

[('wanted submit notice rotation fundamental analysis group thank fx country risk group giving opportunity work exiting learning environment really enjoyed part research group hope keep relationship developed throughout rotation looking forward position opportunity learn company try ability environment fundamental analysis group developing group give opportunity develop grow employee understand responsibility train individual taking current position effort smooth transition rotation train individual starting december th devote free time pursue job responsibility agreement maureen official start date rotation january nd sincerely yana kristal', 0)]
[(OrderedDict([('text', 'wanted submit notice rotation fundamental analysis group thank fx country risk group giving opportunity work exiting learning environment really enjoyed part research group hope keep relationship developed throughout rotation looking forward position opportunity learn company try ability environment fundamental analys

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:439: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:439: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name

--------------------------------------------- Result 1 ---------------------------------------------
[[0 (53%)]] --> [[1 (55%)]]

read holding floor meeting [[following]] date join general overview q session one following floor tuesday location front eb wednesday location front eb thursday location front eb philippe greg disregard email apologize inconvenience

read holding floor meeting [[ensuing]] date join general overview q session one following floor tuesday location front eb wednesday location front eb thursday location front eb philippe greg disregard email apologize inconvenience




/usr/local/lib/python3.7/dist-packages/sklearn/base.py:439: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:439: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, 

--------------------------------------------- Result 2 ---------------------------------------------
[[0 (99%)]] --> [[1 (53%)]]

wanted submit notice rotation fundamental analysis group thank fx country [[risk]] group giving opportunity work exiting learning environment really [[enjoyed]] part research [[group]] hope keep relationship developed throughout rotation looking forward position opportunity learn company try ability environment fundamental analysis group developing group give opportunity develop grow employee understand responsibility train individual taking current position effort smooth transition rotation train individual starting december [[th]] devote free time pursue job responsibility agreement maureen official start [[date]] rotation january nd sincerely yana kristal

wanted submit notice rotation fundamental analysis group thank fx country [[risque]] group giving opportunity work exiting learning environment really [[adores]] part research [[association]] hope keep 

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:439: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:439: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, 

KeyboardInterrupt: ignored